In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, PorterStemmer
import numpy as np
import pandas as pd
import re

In [2]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [3]:
data = pd.read_csv("drive/MyDrive/toxic_classification.csv")
df_0 = data[data['target'] == 0].iloc[:20000, :]
df_1 = data[data['target'] == 1].iloc[:20000, :]
df_small = pd.concat([df_0, df_1])

In [4]:
stemmer = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))
def preprocess(text_string):
    text_string = text_string.lower()
    text_string = re.sub('[^A-Za-z0-9]+', ' ', text_string)
    x = text_string.split()
    new_text = []
    for word in x:
        if word not in stop_words:
            new_text.append(stemmer.stem(word))
    text_string = ' '.join(new_text)
    return text_string

In [5]:
import os.path
import json
if os.path.isfile("drive/MyDrive/small_preprocessX") !=True:
    processed_X = [preprocess(i) for i in df_small.comment_text]
    with open("drive/MyDrive/small_preprocessX", "w") as fp:
        json.dump(processed_X, fp)
else:
    with open('drive/MyDrive/small_preprocessX', "r") as fp:
        processed_X = json.load(fp)

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score,balanced_accuracy_score,precision_score,recall_score
from sklearn.ensemble import BaggingClassifier
from sklearn import linear_model
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression

In [7]:
data_arr = df_small.to_numpy()

In [8]:
#vectorizer = TfidfVectorizer(max_features=1000)
vectorizer = TfidfVectorizer(token_pattern= r'\S+')
X = vectorizer.fit_transform(processed_X)

In [9]:
y = data_arr[:,1].astype('int')

In [10]:
y.shape

(40000,)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [12]:
#pipe = make_pipeline(StandardScaler(with_mean=False), LogisticRegression())
##bagging_LR = BaggingClassifier(pipe,n_estimators=5,max_samples=0.999,verbose=True)
#bagging_LR.fit(X_train, y_train)
#y_bg_LR = bagging_LR.predict(X_test)
#print('Accuracy: %.3f' % accuracy_score(y_test, y_bg_LR))
#print('Balanced Accuracy: %.3f' %balanced_accuracy_score(y_test, y_bg_LR))
#print('Precision:', precision_score(y_test, y_bg_LR))
#print('Recall:', recall_score(y_test, y_bg_LR))
#rint('F1: %.3f' % f1_score(y_test, y_bg_LR))


In [13]:
bagging_SVM = BaggingClassifier(SVC(kernel= 'linear', random_state=1, C=0.1,verbose=True),n_estimators=5,max_samples=0.999,verbose=True)
bagging_SVM.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 12.0min finished


BaggingClassifier(estimator=SVC(C=0.1, kernel='linear', random_state=1,
                                verbose=True),
                  max_samples=0.999, n_estimators=5, verbose=True)

In [14]:
import joblib
# save
#joblib.dump(bagging_SVM, "drive/MyDrive/est3_train10per_SVM.pkl")
joblib.dump(bagging_SVM, "drive/MyDrive/small_est5_train20per_SVM.pkl")

['drive/MyDrive/small_est5_train20per_SVM.pkl']

In [15]:
#import joblib
#bagging_SVM = joblib.load("drive/MyDrive/est3_train10per_SVM.pkl")
#bagging_SVM =joblib.load(bagging_SVM, "drive/MyDrive/small_est5_train20per_SVM.pkl")

In [16]:
#y_bg_svm = np.array([])
#i = 0
#while i < 360975:
   # if i+ 2000 < 360975:
   #   temp = bagging_SVM.predict(X_test[i:i+2000])
   #   y_bg_svm = np.concatenate((y_bg_svm,temp),axis = 0)
   #   i = i + 2000
   # else:
   #   temp = bagging_SVM.predict(X_test[i:360975])
   #   y_bg_svm = np.concatenate((y_bg_svm,temp),axis = 0)
   #   i = 360975
y_bg_svm = bagging_SVM.predict(X_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  4.0min finished


In [17]:
print('Accuracy: %.3f' % accuracy_score(y_test, y_bg_svm))
print('Balanced Accuracy: %.3f' %balanced_accuracy_score(y_test, y_bg_svm))
print('Precision:', precision_score(y_test, y_bg_svm))
print('Recall:', recall_score(y_test, y_bg_svm))
print('F1: %.3f' % f1_score(y_test, y_bg_svm))

Accuracy: 0.843
Balanced Accuracy: 0.843
Precision: 0.8960926193921852
Recall: 0.7747747747747747
F1: 0.831


In [18]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

clf = MLPClassifier(random_state=1,hidden_layer_sizes=(250,25), max_iter=202,early_stopping = True,verbose = True).fit(X_train, y_train)
y_mlp = clf.predict(X_test)
print('Accuracy: %.3f' % accuracy_score(y_test, y_mlp))
print('Balanced Accuracy: %.3f' %balanced_accuracy_score(y_test, y_mlp))
print('Precision:', precision_score(y_test, y_mlp))
print('Recall:', recall_score(y_test, y_mlp))
print('F1: %.3f' % f1_score(y_test, y_mlp))

Iteration 1, loss = 0.45736324
Validation score: 0.855938
Iteration 2, loss = 0.21108018
Validation score: 0.858750
Iteration 3, loss = 0.12688924
Validation score: 0.857187
Iteration 4, loss = 0.08453773
Validation score: 0.850625
Iteration 5, loss = 0.05401152
Validation score: 0.846875
Iteration 6, loss = 0.03405956
Validation score: 0.848437
Iteration 7, loss = 0.02061131
Validation score: 0.848750
Iteration 8, loss = 0.01379278
Validation score: 0.844688
Iteration 9, loss = 0.00832975
Validation score: 0.845938
Iteration 10, loss = 0.00648564
Validation score: 0.847500
Iteration 11, loss = 0.00524869
Validation score: 0.846875
Iteration 12, loss = 0.00510690
Validation score: 0.844688
Iteration 13, loss = 0.00434140
Validation score: 0.842187
Validation score did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
Accuracy: 0.863
Balanced Accuracy: 0.862
Precision: 0.8726196603190942
Recall: 0.8485985985985987
F1: 0.860


In [58]:
X_train, X_test, y_train, y_test,indices_train,indices_test= train_test_split(X, y, range(len(y)), test_size=0.2, random_state=0)

In [60]:
dif_mlp = [index for (index,first, second) in zip(indices_test,y_test, y_mlp) if first != second ]
dif_svm = [index for (index,first, second) in zip(indices_test,y_test, y_bg_svm) if first != second ]

In [113]:
len(dif_svm)

1259

In [114]:
temp = df_small.iloc[dif_mlp]

In [117]:
print("False Positve MLP")
pd.set_option('display.max_rows', None)
print(temp.loc[(temp['target'] == 0)]['comment_text'])
print(len(temp.loc[(temp['target'] == 0)]['comment_text']))

False Positve MLP
13630    I have been voting for the lesser of two evils...
15573    Billy Byron Brown comes to mind, when talking ...
12933                I'm shocked at the demographics. Not!
13051    I hope the dog got a nice treat to get the tas...
13760    I am not talking just about Basketball - you c...
19228    No, it would have to be some modification of a...
19088          At least they're not illegal immigrants :-/
18923    Lance,\nJust hit the ignore button for Wavemak...
7539     Good point, but still had nothing to do with l...
14519    Didn't the State Police officer who ran into a...
11794    Wasilla and Palmer, Alaska should take serious...
16885    Another privileged white person being outraged...
2518     Such is life under the rule of liberalism....t...
616      He is correct. It is legal to kill a cop in se...
18417    I have a 1/2 ton Dodge that would pull the boa...
11460    "...we have an accusation against men for vict...
8288     "Tim" ... the internet is abo

In [118]:
print("False Negative MLP")
print(temp['comment_text'].loc[(temp['target'] == 1)])
print(len(temp.loc[(temp['target'] == 1)]['comment_text']))

False Negative MLP
134002    While that may be true, the 11 homeless people...
115604    I believe "Ward 1" is a woman...and a young on...
66233     Since they reflexively vote republican no matt...
117544    You can tell the story of Susan (the woman in ...
71252          I would have burned the house to the ground.
103596    "If the same common sense that Breyer applied ...
10751     Betty Taylor, notorious hypocrite, voted for t...
212035    Support the election of even more Democrats wi...
250884    Utter and criminally irresponsible lies.  You ...
87047     What in the hell are you talking about? The te...
171710    And refusing to acknowledge ongoing white supr...
17145     Once again you are impeccably ignorant on any ...
19867     Are these legislatures dumb? The only one who ...
7232      No. No.  That's not the way it works.  \n\nThe...
101409    Wow, what poor spelling and grammar you have, ...
45966                    Cop wannabees are generally kooks.
200201               

In [120]:
temp2 = df_small.iloc[dif_svm]

In [124]:
print("False Positve SVM")
print(temp2['comment_text'].loc[(temp2['target'] == 0)])
print(len(temp2.loc[(temp2['target'] == 0)]['comment_text']))

False Positve SVM
13630    I have been voting for the lesser of two evils...
15573    Billy Byron Brown comes to mind, when talking ...
15630         What do you think?  You're the trump fanboi.
13051    I hope the dog got a nice treat to get the tas...
16563                                            Ha ha ha!
17803    At no time do you ever have anything remotely ...
19088          At least they're not illegal immigrants :-/
16501    This is nice and all, but the state already do...
9560     It's time to  play "lawsuit Lottery"!  Yeah, s...
12797    Trump wants to be birther in chief and we know...
18923    Lance,\nJust hit the ignore button for Wavemak...
7845     I guess people could ask the same of Obama, or...
7539     Good point, but still had nothing to do with l...
16885    Another privileged white person being outraged...
8793     He should have waited and shot the cat when it...
20887    the villages should be manning up and paying f...
616      He is correct. It is legal to

In [127]:
print("False Negative SVM")
print(temp2['comment_text'].loc[(temp2['target'] == 1)])
print(len(temp2.loc[(temp2['target'] == 1)]['comment_text']))

False Negative SVM
134002    While that may be true, the 11 homeless people...
126138                  Yes, their both predatory perverts.
208928    George Carlin had a great bit on prostitution ...
66233     Since they reflexively vote republican no matt...
112114    I tried to tell anyone in Eugene that would li...
117544    You can tell the story of Susan (the woman in ...
71252          I would have burned the house to the ground.
125598      So does Bernie, but then again he's a lunatic !
10751     Betty Taylor, notorious hypocrite, voted for t...
16521     Thank you Kathy for having the balls to call o...
245957    You're only partly right, James.\n\nIn middle-...
241185    agreed.  it's a welcomed relief from the incre...
87047     What in the hell are you talking about? The te...
17145     Once again you are impeccably ignorant on any ...
181989    Lane Transit District needs to be neutered bef...
19867     Are these legislatures dumb? The only one who ...
7232      No. No.  Th